In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import joblib

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F

In [2]:
path_to_data = '../external_data/mixmhcpred/TableS2.txt'
# ignore the first row (header)
df = pd.read_csv(path_to_data, sep='\t', skiprows=1)
df['Peptide_Lengths'] = df['Peptide'].apply(len)
df = df[df['Peptide_Lengths'] == 9]

In [ ]:
allele_counts = df['Allele'].value_counts()
# Create a mask to filter IDs that appear at least 10 times
mask = df['Allele'].map(allele_counts) >= 10
df = df[mask]

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 32 # how many independent sequences will we process in parallel?
max_iters = 1500
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 10
dropout = 0.25
vocab_size = 20

# of categories
output_dim = df['Allele'].nunique()

In [12]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [13]:
class pMHCModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.ff = FeedFoward(n_embd)
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, output_dim)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        x = self.token_embedding_table(idx) # (B,T,C)
        x = self.ff(x)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T)
               
     
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        
        return logits, loss

In [14]:
model = pMHCModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e3, 'K parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

2.379 K parameters


In [15]:
label_encoder = LabelEncoder()
# Fit the encoder on your class labels and transform them into numerical labels
y = label_encoder.fit_transform(df['Allele'].values)

In [20]:
amino_acids = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V"]

# create a mapping from amino-acids to integers
stoi_peptide = { ch:i for i,ch in enumerate(amino_acids) }
itos_peptide = { i:ch for i,ch in enumerate(amino_acids) }
encode_peptide = lambda s: [stoi_peptide[c] for c in s] # encoder: take a string, output a list of integers
decode_peptide = lambda l: ''.join([itos_peptide[i] for i in l]) # decoder: take a list of integers, output a string

print(encode_peptide("AAAHTHRY"))
print(decode_peptide(encode_peptide("AAAHTHRY")))

[0, 0, 0, 8, 16, 8, 1, 18]
AAAHTHRY


In [23]:
X = np.array([encode_peptide(sequence) for sequence in df['Peptide'].values])

In [30]:
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(X, dtype=torch.long)
print(data.shape, data.dtype)

ydata = torch.tensor(y, dtype = torch.float)

# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]
train_y = ydata[:n].long()
val_y = ydata[n:].long()

torch.Size([235054, 9]) torch.int64


In [31]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ypred = train_y if split == 'train' else val_y
   
    ix = torch.randint(len(data), (batch_size,))
    x = torch.stack([data[i] for i in ix])
    y = torch.stack([ypred[i] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [32]:
for iter in range(max_iters):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


RuntimeError: shape '[288]' is invalid for input of size 32